# Create a PyQGIS script to calculate the stage volume table
This Jupyter Notebook is part of the course *Programming Basics for QGIS* at the [GIS OpenCourseWare platform](https://courses.gisopencourseware.org/course/view.php?id=80). On the course page you can find instructions and data for this Jupyter Notebook.

In this Jupyter Notebook we'll create a script that loops around the **Raster surface volume** algorithm for creating a table with different levels and their corresponding volumes in a pit.

First we need code to set the parth to the folder where you are storing your data:

In [ ]:
# Set the path to your folder
current_project = QgsProject.instance()
projectPath = current_project.homePath()

Check the result by printing the project path in the cell below.

We can now set the path to the DTM layer in the GeoPackage:

In [ ]:
# Set the path to the DTM layer
inputRasterDTM = "GPKG:" + os.path.join(projectPath,"data_stagevolume.gpkg:DTM")

To load the layer we can use the following code:

In [ ]:
dtmLayer = iface.addRasterLayer(inputRasterDTM,"DTM","gdal")

For the iteration it's important to know the range of the elevation in the DTM. With `bandStatistics(1)` we can calculate statistics of band 1 (in our case we have a single band raster, so we refer to the first band). We can get the lowest value in the DEM with `minimumValue` and the highest elevation with `maximumValue`:

In [ ]:
# Calculate the statistics (min/max) of the DTM
stats = dtmLayer.dataProvider().bandStatistics(1)
dtmMinimum = stats.minimumValue
dtmMaximum = stats.maximumValue
print("min:",dtmMinimum,"m")
print("max:",dtmMaximum,"m")

Now we can easily calculate the range. Complete the code below.

In [ ]:
# Determine the range
dtmRange = # complete this line
print("Elevation Difference:",dtmRange,"m")

Now we can set the number of iterations to 10 and the elevation increment of each iteration to 10% of the range:

In [ ]:
# Set the increment for the iteration at 10% of the range
n_increments = 10
increment = dtmRange / n_increments
print("Increment:",increment)

The next step is to create a *list* of levels to iterate over using Python *list comprehension*.
List comprehension is a compact way to create lists in Python.
Anything you can do with a `for` loop that builds a list, you can usually do with a list comprehension, but in one readable line. Think of it as a mini‑loop written inside square brackets.

In this case you would like to write:


In [ ]:
levels = []
for i in range(n_increments + 1):
    level = demMinimum + i * increment
    levels.append(level)

You loop from i = 0 up to i = n_increments

For each i, you compute a value:
`demMinimum + i * increment`

You append that value to the list.

A more compact way of writing this is with list comprehension:

In [ ]:
levels = [dtmMinimum + i * increment for i in range(n_increments + 1)]

Let's break it into parts:

`demMinimum + i * increment` 
→ the value you want to put in the list

`for i in range(n_increments + 1)`
→ the loop that generates each `i`

*(no condition here, but you could add one if needed)*

So the comprehension says:

“For each i from 0 to n_increments, compute `demMinimum + i * increment` and collect all results in a list.”

Because the *Raster surface volume* algorithm produces a single dbf file for each level, we need to create a list for that:

In [ ]:
# Create an empty list for the dbf files
dbfList = []

Now we need to create a `for` loop to loop over the elevation range from the lowest to the highest elevation with the increment that we have determined before. Within the loop we:

* run the Raster surface volume algorithm with the value for the `LEVEL`
* read the output dbf file
* convert the volumes from m3 to km3
* add the km3 field to the table. Note that we have to import `QVariant` to make this work.
* append the resulting file to `dbfList`



In [ ]:
from qgis.PyQt.QtCore import QVariant

# Loop over the elevation range from the minimum to the maximum with the increment
for level in levels:
    # Define the output table name
    outTable = projectPath +"volume" + str(round(level*100.0))
    outTableDbf = outTable + ".dbf"
    
    # Run the raster surface volume tool with the variables
    processing.run("native:rastersurfacevolume", {
        'INPUT':dtmLayer,
        'BAND':1,
        'LEVEL':level,
        'METHOD':1,
        'OUTPUT_HTML_FILE':'TEMPORARY_OUTPUT',
        'OUTPUT_TABLE':outTable + ".shp"
        })
    
    # Read the table
    dbfTable = QgsVectorLayer(outTableDbf, outTable, "ogr")
    
    # Convert the volumes from m3 to km3
    for feature in dbfTable.getFeatures():
        VolumeKm3 = abs(feature["Volume"])/1000000000.0
    
    # Add the km3 field to the table
    pr = dbfTable.dataProvider()
    pr.addAttributes([QgsField("Level", QVariant.Double),QgsField("VolAbsKm3", QVariant.Double)])
    dbfTable.updateFields()
    dbfTable.startEditing()
    for f in dbfTable.getFeatures():
        f["Level"] = level
        f["VolAbsKm3"] = VolumeKm3
        dbfTable.updateFeature(f)
    dbfTable.commitChanges()
 
    dbfList.append(outTableDbf)

You can check in the *Browser* panel to see that this resulted in many separate dbf files. We now need to mege all dbf files into one. The QGIS algorithm *Merge vector layers* can be used for that:

In [ ]:
# Merge all dbf files into one
processing.run("native:mergevectorlayers", {
    'LAYERS':dbfList,
    'CRS':None,
    'OUTPUT':projectPath + 'stagevolume.shp'
    })
vlayer = QgsVectorLayer(projectPath + 'stagevolume.dbf', "StageVolume", "ogr")

Now we can add the resulting table to the project:

In [ ]:
# Add the result to the project
QgsProject.instance().addMapLayer(vlayer)

Open the table as an attribute table and check the results.
Now you can follow the next steps in the tutorial at GIS OpenCourseWare to create a stage-volume curve with the Data Plotly plugin, based on this table.